# Machine Learning Portfolio Example

This example illustrates how to use all Chen-Zimmermann predictors, together with CRSP data. We'll use the `openassetpricing` package to download the Chen-Zimmermann predictors, merge with monthly CRSP, fit returns to lagged signals, and form portfolios in out-of-sample tests. Specifically, we'll use a "groovy" model (fit on the 1960s and 1970s) to try to predict returns during hair metal (1980s), gangsta rap (1990s), emo (2000s), and other more recent samples. Does the groovy model work even in the TSwift (and that Chiefs guy) era?

Downloading all of the signals takes some time and requires substantial RAM. It also requires a WRDS account, since some predictors require data from WRDS (size, short-term reversal, price). 

To manage memory, we'll use Polars dataframes instead of Pandas, and set signal variables to float32 (instead of float64). 

In [67]:
import pandas as pd
import polars as pl
import openassetpricing as oap
import numpy as np
import wrds
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Initialize OpenAP
openap = oap.OpenAP()

<span style="color:orange;font-weight:bold">Optional:</span>
By default, we'll use all 212 signals for forming portfolios. This takes about 35 minutes on a desktop with with 64 GB of RAM. 

You may want to reduce `nsignals_for_ml` for a faster run or a smaller machine. We found `nsignals_for_ml = 20` runs quickly on a laptop with 16 GB of RAM. But it won't reproduce the nice results in [Gu, Kelly, Xiu (2020)](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3159577); [Chen and McCoy (2024)](https://arxiv.org/abs/2207.13071); and elsewhere.

In [68]:
# User-specified number of signals for ML
nsignals_for_ml = 212

# Download data

You'll have to enter your WRDS credentials twice: once to download the CRSP returns, and once to download all Chen-Zimmermann predictors (including size, short-term reversal, and price). The downloads take a couple minutes in total. We keep only standard common stocks on major exchanges (see https://github.com/OpenSourceAP/CrossSection/issues/133).

In [70]:
wrds_conn = wrds.Connection()

crsp = wrds_conn.raw_sql(
    """select a.permno, a.date, a.ret*100 as ret
                        from crsp.msf a
                        join crsp.msenames b 
                        on a.permno = b.permno
                        and a.date >= b.namedt
                        and a.date <= b.nameendt
                        where b.shrcd in (10, 11, 12) 
                        and b.exchcd in (1, 2, 3)""",
    date_cols=["date"],
)

crsp = pl.from_pandas(crsp)

WRDS recommends setting up a .pgpass file.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [71]:
# Download all Chen-Zimmermann predictors
bigdat = openap.dl_all_signals("polars")

WRDS recommends setting up a .pgpass file.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done

Data is downloaded: 2 mins


In [72]:
# Get names of all signals
signal_list = [col for col in bigdat.columns if col not in ["permno", "yyyymm"]]

bigdat = bigdat.with_columns(pl.col(signal_list).cast(pl.Float32))

# show MSFT (permno = 10107)
bigdat.filter(pl.col("permno") == 10107).head(24)

permno,yyyymm,AM,AOP,AbnormalAccruals,Accruals,AccrualsBM,Activism1,Activism2,AdExp,AgeIPO,AnalystRevision,AnalystValue,AnnouncementReturn,AssetGrowth,BM,BMdec,BPEBM,Beta,BetaFP,BetaLiquidityPS,BetaTailRisk,BidAskSpread,BookLeverage,BrandInvest,CBOperProf,CF,CPVolSpread,Cash,CashProd,ChAssetTurnover,ChEQ,ChForecastAccrual,ChInv,ChInvIA,ChNAnalyst,ChNNCOA,…,Spinoff,SurpriseRD,Tax,TotalAccruals,TrendFactor,UpRecomm,VarCF,VolMkt,VolSD,VolumeTrend,XFIN,betaVIX,cfp,dCPVolSpread,dNoa,dVolCall,dVolPut,fgr5yrLag,grcapx,grcapx3y,hire,iomom_cust,iomom_supp,realestate,retConglomerate,roaq,sfe,sinAlgo,skew1,std_turn,tang,zerotrade12M,zerotrade1M,zerotrade6M,Price,Size,STreversal
i32,i32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
10107,198603,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,1.0,null,null,null,0.242157,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,-3.314186,-6.553609,-0.0
10107,198604,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.004334,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,1.0,null,null,null,0.193979,null,null,null,null,null,null,-0.015511,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.4361e-8,null,-3.473518,-6.712941,-0.172727
10107,198605,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.004821,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,1.0,null,null,null,0.159634,null,null,null,null,null,null,-0.013768,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2.6631e-8,null,-3.54818,-6.787603,-0.077519
10107,198606,null,null,null,null,null,null,null,null,11.0,null,null,null,null,null,null,null,null,null,null,null,0.007609,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,1.0,null,null,null,0.1693,null,null,null,null,null,null,0.017796,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,5.4195e-8,null,-3.42589,-6.665352,0.115108
10107,198607,null,null,null,null,null,null,null,null,11.0,null,null,0.036261,null,null,null,null,null,null,null,null,0.006991,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,1.0,null,null,null,0.152681,null,null,null,null,null,null,0.003951,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,5.1931e-8,null,-3.349904,-6.589366,0.073171
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
10107,198710,0.064755,-2.042913,null,null,null,null,null,0.005479,12.0,0.509589,0.547472,-0.09505,null,-1.728493,null,0.034875,null,null,null,null,0.014662,-1.225411,null,0.309139,0.017049,null,0.442655,-24.01692,null,null,0.0,null,null,null,null,…,1.0,null,1.971073,null,0.286837,null,null,-0.231705,null,null,-0.282056,-0.000679,null,null,null,null,null,-18.17,null,null,0.0,-22.978674,-24.611324,-0.041296,null,0.066585,null,null,null,null,null,3.3413e-8,1.0079e-8,5.9255e-8,-3.907011,-7.877284,0.249057
10107,198711,0.07199,-2.042913,null,null,null,null,null,0.006091,12.0,1.016129,0.547472,-0.09505,null,-1.728493,null,0.038372,1.5192,null,null,null,0.012299,-1.225411,null,0.309139,0.018954,null,0.442655,-21.436035,null,null,1.0,null,null,null,null,…,1.0,null,1.971073,null,0.327159,null,null,-0.268145,null,null,-0.282056,0.000342,null,null,null,null,null,

Above gives you a sense of what the data looks like. First two columns are identifiers. The other columns are the values of the signals. There are a lot of missing values for MSFT (permno = 10107) when it first listed, back in 1986. But by 1987, a lot of these values are filled in. This is a common missingness pattern for accounting predictors ([Chen and McCoy (2024)](https://arxiv.org/abs/2207.13071); also [these discussion slides](https://drive.google.com/file/d/1e8aGG9JtufQKeA_jMZ3XaP6wNhJS6Y0q/view)).

# Lag signals and merge

To lag signals, you can just add one month to the `yyyymm` column. For simplicity, let's fill in the day of the new variable `date` as the 28th (the signals are assumed to be available for trading at the end of the month). You can keep around `yyyymm` as `yyyymm_signals` for sanity checks.

In [73]:
bigdat = bigdat.select(
    "permno",
    # Create date that is one month ahead for merging with returns
    pl.col("yyyymm")
    .cast(pl.String)
    .add("28")
    .str.to_date("%Y%m%d")
    .dt.offset_by("1mo")
    .alias("date"),
    # rename yyyymm for clarity
    pl.col("yyyymm").alias("yyyymm_signals"),
    pl.col(signal_list),
)

bigdat.head()

permno,date,yyyymm_signals,AM,AOP,AbnormalAccruals,Accruals,AccrualsBM,Activism1,Activism2,AdExp,AgeIPO,AnalystRevision,AnalystValue,AnnouncementReturn,AssetGrowth,BM,BMdec,BPEBM,Beta,BetaFP,BetaLiquidityPS,BetaTailRisk,BidAskSpread,BookLeverage,BrandInvest,CBOperProf,CF,CPVolSpread,Cash,CashProd,ChAssetTurnover,ChEQ,ChForecastAccrual,ChInv,ChInvIA,ChNAnalyst,…,Spinoff,SurpriseRD,Tax,TotalAccruals,TrendFactor,UpRecomm,VarCF,VolMkt,VolSD,VolumeTrend,XFIN,betaVIX,cfp,dCPVolSpread,dNoa,dVolCall,dVolPut,fgr5yrLag,grcapx,grcapx3y,hire,iomom_cust,iomom_supp,realestate,retConglomerate,roaq,sfe,sinAlgo,skew1,std_turn,tang,zerotrade12M,zerotrade1M,zerotrade6M,Price,Size,STreversal
i32,date,i32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
10000,1986-02-28,198601,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,0.0,null,null,null,null,null,null,null,null,null,null,-0.005234,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,-1.475906,-2.778819,-0.0
10000,1986-03-28,198602,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,0.0,null,null,null,null,null,null,null,null,null,null,-0.003488,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,4.7852e-8,null,-1.178655,-2.481568,0.257143
10000,1986-04-28,198603,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,0.0,null,null,null,null,null,null,null,null,null,null,-0.002715,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0234e-7,null,-1.490091,-2.793004,-0.365385
10000,1986-05-28,198604,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,0.0,null,null,null,null,null,null,null,null,null,null,0.000877,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.4675e-8,null,-1.386294,-2.719452,0.098592
10000,1986-06-28,198605,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,0.0,null,null,null,null,null,null,null,null,null,null,0.008817,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.6496e-8,null,-1.134423,-2.467581,0.222656


Now merge with CRSP. Convert CRSP dates to the 28th of the month for simplicity. The left join makes the missing values issues transparent.

In [74]:
# Convert crsp dates to the 28th of the month
crsp = crsp.select(
    pl.col("permno").cast(pl.Int32),
    pl.col("date")
    .dt.year()
    .mul(100)
    .add(pl.col("date").dt.month())
    .cast(pl.String)
    .add("28")
    .str.to_date("%Y%m%d"),
    "ret",
)

# lLeft join returns onto signals, in-place (for ram)
bigdat = crsp.join(bigdat, on=["permno", "date"], how="left")

bigdat.head()

permno,date,ret,yyyymm_signals,AM,AOP,AbnormalAccruals,Accruals,AccrualsBM,Activism1,Activism2,AdExp,AgeIPO,AnalystRevision,AnalystValue,AnnouncementReturn,AssetGrowth,BM,BMdec,BPEBM,Beta,BetaFP,BetaLiquidityPS,BetaTailRisk,BidAskSpread,BookLeverage,BrandInvest,CBOperProf,CF,CPVolSpread,Cash,CashProd,ChAssetTurnover,ChEQ,ChForecastAccrual,ChInv,ChInvIA,…,Spinoff,SurpriseRD,Tax,TotalAccruals,TrendFactor,UpRecomm,VarCF,VolMkt,VolSD,VolumeTrend,XFIN,betaVIX,cfp,dCPVolSpread,dNoa,dVolCall,dVolPut,fgr5yrLag,grcapx,grcapx3y,hire,iomom_cust,iomom_supp,realestate,retConglomerate,roaq,sfe,sinAlgo,skew1,std_turn,tang,zerotrade12M,zerotrade1M,zerotrade6M,Price,Size,STreversal
i32,date,f64,i32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
10000,1986-01-28,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
10000,1986-02-28,-25.7143,198601,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,0.0,null,null,null,null,null,null,null,null,null,null,-0.005234,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,-1.475906,-2.778819,-0.0
10000,1986-03-28,36.5385,198602,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,0.0,null,null,null,null,null,null,null,null,null,null,-0.003488,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,4.7852e-8,null,-1.178655,-2.481568,0.257143
10000,1986-04-28,-9.8592,198603,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,0.0,null,null,null,null,null,null,null,null,null,null,-0.002715,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0234e-7,null,-1.490091,-2.793004,-0.365385
10000,1986-05-28,-22.2656,198604,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,0.0,null,null,null,null,null,null,null,null,null,null,0.000877,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.4675e-8,null,-1.386294,-2.719452,0.098592


Congrats, the data is merged! But unfortunately, we'll need to do a bit more work to make it usable.

# Process data for ML
We'll need to deal with the missing signals. This is a notorious issue with big data. Here, we'll just standardize the signals and then fill in missings with zero. This follows [Chen and McCoy (2024)](https://arxiv.org/abs/2207.13071).

In [75]:
# Copy over, keep only after 1963 and non-missing returns
cleandat = bigdat.filter(pl.col("date").dt.year() >= 1963, pl.col("ret").is_not_null())

cleandat = (
    cleandat
    # Standardize
    .with_columns(
        (pl.col(signal_list) - pl.col(signal_list).mean()) / pl.col(signal_list).std()
    )
    # Replace null with 0
    .fill_null(0)
)

<span style="color:orange;font-weight:bold">Optional:</span> Below, we reduce the number of signals for tractability. To use more signals, modify `nsignals_for_ml` at the beginning of the notebook.

In [76]:
# make data smaller
nsignals_for_ml = min(len(signal_list), nsignals_for_ml)
# Select columns from cleandat: (permno, date, ret, yyyymm_signals, [nsignals_for_ml])
cleandat = cleandat.select(cleandat.columns[:(4 + nsignals_for_ml)])
# Make our signal list match our selected data
signal_list = signal_list[:nsignals_for_ml]

# Show clean data with filtered/unfiltered signal list
cleandat.head()

permno,date,ret,yyyymm_signals,AM,AOP,AbnormalAccruals,Accruals,AccrualsBM,Activism1,Activism2,AdExp,AgeIPO,AnalystRevision,AnalystValue,AnnouncementReturn,AssetGrowth,BM,BMdec,BPEBM,Beta,BetaFP,BetaLiquidityPS,BetaTailRisk,BidAskSpread,BookLeverage,BrandInvest,CBOperProf,CF,CPVolSpread,Cash,CashProd,ChAssetTurnover,ChEQ,ChForecastAccrual,ChInv,ChInvIA,…,Spinoff,SurpriseRD,Tax,TotalAccruals,TrendFactor,UpRecomm,VarCF,VolMkt,VolSD,VolumeTrend,XFIN,betaVIX,cfp,dCPVolSpread,dNoa,dVolCall,dVolPut,fgr5yrLag,grcapx,grcapx3y,hire,iomom_cust,iomom_supp,realestate,retConglomerate,roaq,sfe,sinAlgo,skew1,std_turn,tang,zerotrade12M,zerotrade1M,zerotrade6M,Price,Size,STreversal
i32,date,f64,i32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
10000,1986-02-28,-25.7143,198601,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,-0.160952,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.25426,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.644967,0.851256,0.059751
10000,1986-03-28,36.5385,198602,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,-0.160952,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.159584,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.413432,0.0,0.870564,0.979924,1.458834
10000,1986-04-28,-9.8592,198603,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,-0.160952,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.117701,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.413432,0.0,0.634201,0.845117,-1.928263
10000,1986-05-28,-22.2656,198604,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,-0.160952,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.07704,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.413432,0.0,0.712977,0.876954,0.596178
10000,1986-06-28,-0.5025,198605,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,-0.160952,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.507541,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.413432,0.0,0.904133,0.985979,1.271195


# ML-style portfolios (with OLS)
Following Lewellen (2014, CFR), let's predict returns using many signals and then sort stocks on the predicted returns. We'll do this in perhaps the simplest way possible: fit returns with OLS using the "groovy" 1963-1979 sample. Then use the fitted coefficients on lagged signals to sort stocks every month from 1980 onward. 

This can't work, can it?

In [77]:
# User-specified fit period
fit_start = 1963
fit_end = 1979

# User-specified number of portfolios
nport = 5

# Make a copy of our dataframe specific to the OLS example
cleandat_ols = cleandat

In [78]:
# Fit returns
formula = "ret ~ " + " + ".join(signal_list)

fit = smf.ols(
    formula,
    data=cleandat_ols.filter(
        pl.col("date").dt.year().is_between(fit_start, fit_end - 1)
    ),
).fit()

In [79]:
# Apply fit to all data
# Do it chunk by chunk to avoid large momory consumption caused by large matrix operation
res = []
for i in cleandat_ols.iter_slices(n_rows=len(cleandat_ols) // 100):
    temp = i.select(pl.lit(1), pl.col(signal_list)).to_numpy() @ fit.params.values
    res += list(temp)

cleandat_ols = cleandat_ols.with_columns(pred=np.array(res))

In [80]:
# == Find portfolio returns ==

# Copy data
preddat = cleandat_ols.select("permno", "date", "pred", "ret").to_pandas()

# Define port sort function
# Follows https://github.com/chenandrewy/flex-mining/blob/70ca658090a13fea8517945280b2de83b9886968/0_Environment.R#L465
def port_sort(x, nport):
    return np.ceil(x.rank(method="min") * nport / (len(x) + 1)).astype(int)

preddat["port"] = preddat.groupby("date")["pred"].transform(port_sort, nport=nport)

# Find portfolio returns
portdat = (
    preddat.groupby(["port", "date"], observed=False)
    .agg(ret=("ret", "mean"), nstock=("permno", "nunique"))
    .reset_index()
)

How does this portfolio do after the "groovy" era? Let's check how it does during the hair metal (1980s), gangsta rap (1990s), emo (2000s), EDM (2010s), and TSwift (2020s) samples.

In [81]:
# Find performance by 10-year periods
samplength = 10

portdat["subsamp"] = pd.cut(
    portdat["date"].dt.year,
    bins=range(1959, 2030, samplength),
    labels=range(1959, 2029, samplength),
)

portsum = (
    portdat.groupby(["port", "subsamp"], observed=False)
    .agg(
        meanret=("ret", "mean"),
        vol=("ret", "std"),
        nmonth=("date", "nunique"),
        nstock=("nstock", "mean"),
        datemin=("date", "min"),
        datemax=("date", "max"),
    )
    .reset_index()
)
portsum["meanret"] = round(portsum["meanret"], 2)

# Pivot meanret to wide format
sumwide = portsum.pivot(index="subsamp", columns="port", values="meanret").reset_index()
sumwide.columns = ["subsamp"] + [f"port_{col}" for col in sumwide.columns[1:]]

# Add long-short
sumwide["5_minus_1"] = sumwide["port_5"] - sumwide["port_1"]

# Add date ranges
temp = (
    portsum.groupby("subsamp", observed=False)
    .agg(datemin=("datemin", "min"), datemax=("datemax", "max"))
    .reset_index()
)

sumwide = pd.merge(temp, sumwide, on="subsamp", how="left")

# Name the subsamples
sumwide["subsamp"] = sumwide["subsamp"].map(
    {
        1959: "groovy",
        1969: "groovy (still)",
        1979: "hair metal",
        1989: "gangsta rap",
        1999: "emo",
        2009: "EDM",
        2019: "TSwift",
    }
)

sumwide

,subsamp,datemin,datemax,port_1,port_2,port_3,port_4,port_5,5_minus_1
0,groovy,1963-01-28,1969-12-28,0.49,0.99,1.44,1.90,2.92,2.43
1,groovy (still),1970-01-28,1979-12-28,-0.82,0.33,1.07,1.87,3.29,4.11
2,hair metal,1980-01-28,1989-12-28,-0.10,0.91,1.34,1.58,2.35,2.45
3,gangsta rap,1990-01-28,1999-12-28,-0.00,0.73,1.12,1.69,3.54,3.54
4,emo,2000-01-28,2009-12-28,-0.24,0.38,0.84,1.22,2.57,2.81
5,EDM,2010-01-28,2019-12-28,0.42,0.84,0.93,1.12,1.40,0.98
6,TSwift,2020-01-28,2023-12-28,-0.16,0.82,0.89,1.05,1.36,1.52


The OLS model, fitted only using groovy era data, makes it through hair metal, gansta rap, and emo quite well. In the corresponding decades, the groovy model earns long-short returns of 2.0 to 3.0 percent per month. So a model from the [Simon and Garfunkel](https://en.wikipedia.org/wiki/Groovy#/media/File:Soundofsilence.jpg) days continued to predict quite well, even while [Metallica inexplicably started to paint their fingernails black](https://www.reddit.com/r/Metallica/comments/huk18i/never_forget_emotallica/). 

During EDM and the Tswift eras, the model produces some notable magnitudes, though the returns are much weaker than they were while [Ms. Swift was still into pickup trucks](https://www.youtube.com/watch?v=GkD20ajVxnY).

There are huge caveats about trading costs (Chen and Velikov 2023). But then again, this tutorial doesn't even attempt to deal with trading costs. One can likely do much better by following DeMiguel, Martin-Utrera, Nogales, and Uppal (2020) or Jensen, Kelly, Malamud, and Pedersen (2024).

# ML-Portfolios with a Simple Feed-Forward Neural Network 

It's kind of cringey to call OLS "machine learning" (almost as cringey as [Emotallica](https://www.reddit.com/r/Metallica/comments/huk18i/never_forget_emotallica/)). Let's make a proper ML portfolio, using now with scikit-learn's implementation of a simple neural network. 

In [82]:
# User-specified parameters
fit_start = 1963
fit_end = 1979
nport = 5

cleandat_mlp = cleandat

In [83]:
# Prepare data for training
filtered_data = cleandat_mlp.filter(
    pl.col("date").dt.year().is_between(fit_start, fit_end - 1)
)
X = filtered_data.select(pl.col(signal_list)).to_pandas().values
y = filtered_data.select(pl.col("ret")).to_pandas().values.ravel()

In [84]:
# Standardize features for neural network
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [85]:
# Train/test split for evaluation
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=1
)

# Define and train the MLPRegressor
mlp = MLPRegressor(
    hidden_layer_sizes=(32, 16, 8),  # Increase neurons for more learning capacity
    activation="relu",  # Good for nonlinear relationships
    solver="adam",  # Robust optimizer for noisy data
    alpha=0.001,  # Add regularization to reduce overfitting
    batch_size=10000,
    learning_rate_init=0.01,  # Lower learning rate for finer optimization
    max_iter=100,  # Allow more iterations for convergence
    early_stopping=True,  # Stop training if validation error doesn't improve
    n_iter_no_change=5,  # Patience for early stopping
    random_state=1,  # Ensure reproducibility
)
mlp.fit(X_train, y_train)

MLPRegressor(alpha=0.001, batch_size=10000, early_stopping=True,
             hidden_layer_sizes=(32, 16, 8), learning_rate_init=0.01,
             max_iter=100, n_iter_no_change=5, random_state=1)

In [86]:
# Apply the trained model to all data (chunk by chunk)
res = []
for i in cleandat_mlp.iter_slices(n_rows=len(cleandat_mlp) // 100):
    temp_data = i.select(pl.col(signal_list)).to_numpy()
    temp_data_scaled = scaler.transform(temp_data)
    temp_mlp = mlp.predict(temp_data_scaled)
    res.extend(temp_mlp)

cleandat_mlp = cleandat_mlp.with_columns(pred=np.array(res))

In [87]:
# == Find portfolio returns ==
# Copy data
preddat_mlp = cleandat_mlp.select("permno", "date", "pred", "ret").to_pandas()

# Define port sort function
def port_sort(x, nport):
    return np.ceil(x.rank(method="min") * nport / (len(x) + 1)).astype(int)

preddat_mlp["port"] = preddat_mlp.groupby("date")["pred"].transform(
    port_sort, nport=nport
)

# Find portfolio returns
portdat_mlp = (
    preddat_mlp.groupby(["port", "date"], observed=False)
    .agg(ret=("ret", "mean"), nstock=("permno", "nunique"))
    .reset_index()
)

Okay, now that we have our portfolio, fitted to groovy data. Let's see how it works on more recent data, from hair metal to Taylor and the Chiefs guy.

In [88]:
# Find performance by 10-year periods
samplength = 10

portdat_mlp["subsamp"] = pd.cut(
    portdat_mlp["date"].dt.year,
    bins=range(1959, 2030, samplength),
    labels=range(1959, 2029, samplength),
)

portsum_mlp = (
    portdat_mlp.groupby(["port", "subsamp"], observed=False)
    .agg(
        meanret=("ret", "mean"),
        vol=("ret", "std"),
        nmonth=("date", "nunique"),
        nstock=("nstock", "mean"),
        datemin=("date", "min"),
        datemax=("date", "max"),
    )
    .reset_index()
)
portsum_mlp["meanret"] = round(portsum_mlp["meanret"], 2)

# Pivot meanret to wide format
sumwide_mlp = portsum_mlp.pivot(
    index="subsamp", columns="port", values="meanret"
).reset_index()
sumwide_mlp.columns = ["subsamp"] + [f"port_{col}" for col in sumwide_mlp.columns[1:]]

# Add long-short
sumwide_mlp["5_minus_1"] = sumwide_mlp["port_5"] - sumwide_mlp["port_1"]

# Add date ranges
temp_mlp = (
    portsum_mlp.groupby("subsamp", observed=False)
    .agg(datemin=("datemin", "min"), datemax=("datemax", "max"))
    .reset_index()
)

sumwide_mlp = pd.merge(temp_mlp, sumwide_mlp, on="subsamp", how="left")

# Name the subsamples
sumwide_mlp["subsamp"] = sumwide_mlp["subsamp"].map(
    {
        1959: "groovy",
        1969: "groovy (still)",
        1979: "hair metal",
        1989: "gangsta rap",
        1999: "emo",
        2009: "EDM",
        2019: "TSwift",
    }
)

sumwide_mlp

,subsamp,datemin,datemax,port_1,port_2,port_3,port_4,port_5,5_minus_1
0,groovy,1963-01-28,1969-12-28,0.03,0.94,1.35,1.98,3.45,3.42
1,groovy (still),1970-01-28,1979-12-28,-1.48,-0.11,1.01,2.01,4.32,5.80
2,hair metal,1980-01-28,1989-12-28,0.25,0.96,1.30,1.49,2.08,1.83
3,gangsta rap,1990-01-28,1999-12-28,0.34,0.75,1.20,1.61,3.17,2.83
4,emo,2000-01-28,2009-12-28,0.03,0.44,0.83,1.12,2.34,2.31
5,EDM,2010-01-28,2019-12-28,0.66,0.96,0.96,1.06,1.06,0.40
6,TSwift,2020-01-28,2023-12-28,0.51,0.46,0.72,1.19,1.08,0.57


Ew. 

Our neural network actually performs somewhat worse than OLS. While Tommy and Gina were [Livin on a Prayer](https://www.youtube.com/watch?v=lDK9QqIzhwk), or Snoop was [sippin on Gin and Juice](https://www.youtube.com/watch?v=fWCZse1iwE0), the neural network earns about 20% less than OLS. 

This might be because we're not following the best practices from the literature. You can't just fit a model on some hippie stuff and expect to work after Reagan takes office. 

The best practice is to refit frequently, to ensure that the model is up-to-date (e.g. [Gu, Kelly, Xiu 2020](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3159577)). 

# Neural Network With Refitting
Let's use expanding windows with 2 years in between each refitting

In [91]:
# Parameters for expanding window estimation
cleandat_refit = cleandat  # Copy of clean data
refit_period = 2 # Number of years between model refits
fit_start = 1963  # Initial training start year
fit_end = 1979  # Initial training end year
nport = 5  # Number of portfolios to form

# Lists to store results
all_predictions = []  # Store all model predictions
all_portdat = []  # Store all portfolio returns

# Rolling window estimation
for end_year in range(fit_end, 2030, refit_period):
    # Get training data between 1963-1979
    train_data = cleandat_refit.filter(
        pl.col("date").dt.year().is_between(fit_start, end_year)
    )
    X_train = train_data.select(pl.col(signal_list)).to_pandas().values
    y_train = train_data.select(pl.col("ret")).to_pandas().values.ravel()
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    # Initialize and train neural network
    mlp = MLPRegressor(
        hidden_layer_sizes=(32, 16, 8),
        activation="relu",
        solver="adam",
        alpha=0.0001,
        batch_size=10000,
        learning_rate_init=0.01,
        max_iter=100,
        early_stopping=True,
        n_iter_no_change=5,
        random_state=end_year,
    )
    mlp.fit(X_train_scaled, y_train)

    # Get test data for next period
    test_data = cleandat_refit.filter(
        pl.col("date").dt.year().is_between(end_year + 1, end_year + refit_period)
    )
    if len(test_data) == 0:
        continue

    # Generate predictions on test data
    X_test = test_data.select(pl.col(signal_list)).to_pandas().values
    X_test_scaled = scaler.transform(X_test)
    predictions = mlp.predict(X_test_scaled)

    # Add predictions to test data
    test_data = test_data.with_columns(pred=predictions).select(
        ["permno", "date", "pred", "ret"]
    )
    all_predictions.append(test_data)

    # Form portfolios based on predictions
    preddat_refit = test_data.to_pandas()
    preddat_refit["port"] = preddat_refit.groupby("date")["pred"].transform(
        port_sort, nport=nport
    )

    # Calculate portfolio returns
    portdat_refit = (
        preddat_refit.groupby(["port", "date"], observed=False)
        .agg(ret=("ret", "mean"), nstock=("permno", "nunique"))
        .reset_index()
    )
    all_portdat.append(portdat_refit)

# Combine results
all_predictions = pl.concat(all_predictions).select(["permno", "date", "pred", "ret"])
portdat_refit = pd.concat(all_portdat, ignore_index=True)

# Create subsamples for analysis
samplength = 10

portdat_refit["subsamp"] = pd.cut(
    portdat_refit["date"].dt.year,
    bins=range(1959, 2030, samplength),
    labels=range(1959, 2029, samplength),
)

# Calculate summary statistics by portfolio and subsample
portsum_refit = (
    portdat_refit.groupby(["port", "subsamp"], observed=False)
    .agg(
        meanret=("ret", "mean"),
        vol=("ret", "std"),
        nmonth=("date", "nunique"),
        nstock=("nstock", "mean"),
        datemin=("date", "min"),
        datemax=("date", "max"),
    )
    .reset_index()
)
portsum_refit["meanret"] = round(portsum_refit["meanret"], 2)

# Pivot results to wide format
sumwide_refit = portsum_refit.pivot(
    index="subsamp", columns="port", values="meanret"
).reset_index()
sumwide_refit.columns = ["subsamp"] + [
    f"port_{col}" for col in sumwide_refit.columns[1:]
]

# Calculate long-short portfolio returns
sumwide_refit["5_minus_1"] = sumwide_refit["port_5"] - sumwide_refit["port_1"]

# Add date range for each subsample
temp_refit = (
    portsum_refit.groupby("subsamp", observed=False)
    .agg(datemin=("datemin", "min"), datemax=("datemax", "max"))
    .reset_index()
)

sumwide_refit = pd.merge(temp_refit, sumwide_refit, on="subsamp", how="left")

# Add the labels for each decade
sumwide_refit["subsamp"] = sumwide_refit["subsamp"].map(
    {
        1959: "groovy",
        1969: "groovy (still)",
        1979: "hair metal",
        1989: "gangsta rap",
        1999: "emo",
        2009: "EDM",
        2019: "TSwift",
    }
)

# Drop first two rows (groovy & groovy (still))
# This is because we are refitting with expanding windows so predictions do not exis for the test data 
sumwide_refit = sumwide_refit.dropna()

sumwide_refit

,subsamp,datemin,datemax,port_1,port_2,port_3,port_4,port_5,5_minus_1
2,hair metal,1980-01-28,1989-12-28,-0.34,0.77,1.35,1.79,2.51,2.85
3,gangsta rap,1990-01-28,1999-12-28,-0.94,0.51,1.26,2.06,4.21,5.15
4,emo,2000-01-28,2009-12-28,-1.21,0.23,1.08,1.59,3.13,4.34
5,EDM,2010-01-28,2019-12-28,-0.11,0.88,1.01,1.23,1.97,2.08
6,TSwift,2020-01-28,2023-12-28,-0.57,0.69,0.81,1.30,1.98,2.55


Ah, that's more like it. Refitting every 2 years, the neural network consistently outperforms OLS, with returns similar to those from [Gu, Kelly, Xiu (2020)](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3159577) and [Chen and McCoy (2024)](https://arxiv.org/abs/2207.13071). 

You kinda wish it was better than this. I mean, neural networks are supposed to solve general intelligence and everything. But if you read between the lines in the literature, you might see some other results that resemble these.

Still, it's instructive to understand how important refitting is. It's natural to think the stock market predictability is not stable (e.g. McLean-Pontiff 2016; Chen-Velikov 2023). So you need to keep updating the model, to avoid using stale information. This helps explain why the groovy model fails to perform in the emo period. 

But why Metallica started to paint their fingernails black... ...remains a mystery.